In [9]:
import time, os, csv
import json, re, random
from requests import session
from lxml.html import fromstring
from fake_useragent import UserAgent
from DrissionPage import ChromiumOptions
from DrissionPage.common import Keys
from DrissionPage import Chromium, ChromiumPage
ua = UserAgent()

In [10]:
browser = ChromiumPage()
time.sleep(2)

option = ChromiumOptions()
driver = browser.new_tab()


# option.headless()

In [ ]:

page_url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco%2C+CA&start=0'


In [ ]:
pg = 13

while True:
    item_per_page = 10
    page = pg * item_per_page
    page_url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco%2C+CA&start={page}&dd_referrer='
    print(page_url)
    driver.get(page_url)
    driver._wait_loaded(5)
    [driver.actions.key_down(k) or time.sleep(1.7) for k in [Keys.DOWN]* 86]
    soup = fromstring(driver.html)
    contaier = soup.xpath('//div[@data-testid="serp-ia-card"]//div[contains(@class,"businessName")]//a')
    nextPg = soup.xpath('//button[contains(@class, "pagination-button") and @disabled]')
    if nextPg:break
    for item in contaier:
        href_link = 'https://www.yelp.com' + item.get('href')
        if 'biz' in href_link:
            print(f'[*] Business Links : {href_link}')
            with open('yelp_links.txt', 'a', encoding='utf-8') as file:
                file.write(href_link + '\n')
    pg += 1
    

In [ ]:
soup = fromstring(driver.html)
nextPage = '//div[@role="navigation"]//span[text() = "Next Page"]'

In [5]:
contaier = soup.xpath('//div[@data-testid="serp-ia-card"]//div[contains(@class,"businessName")]//a')

In [6]:
print(len(contaier))
data_links = []

14


In [1]:
from urllib.parse import urlencode
page = 0
params = {
        "find_desc": "Restaurants",
        "find_loc": "San Francisco, CA",
        "start": page,
        "dd_referrer": ""
    }
page_url = "https://www.yelp.com/search?" + urlencode(params)


print(page_url)

https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco%2C+CA&start=0&dd_referrer=


In [ ]:
open('sample.txt', 'w', encoding='utf-8').close()


with open('sample.txt', encoding='utf-8') as file:
    saved_links = [link.strip() for link in file.readlines()]


new_links = [
    'https://www.yelp.com/adredir?ad_business_id=Aapgx3LzavPPp1g1KXW8uQ&campaign_id=212670200&click_origin=search_results&placement=vertical_0&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fbetter-than-sweet-san-francisco&request_id=1b572cc9c74ee95a&signature=37941a0df69c958f9a470cbc42dad45f2103a5c03b854bb35115e44f7c076179&slot=0',
    'https://www.yelp.com/adredir?ad_business_id=JA1DjwFEJ97PrsqA4QKPWA&campaign_id=210274275&click_origin=search_results&placement=vertical_0&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fhalal-city-san-francisco&request_id=1b572cc9c74ee95a&signature=f8123a6c46024122bc74e4d72f41428aadf0c31f86b93aa402ea20f41d7ccbdf&slot=1',
    'https://www.yelp.com/biz/bottega-san-francisco-2?osq=Restaurants'
]
unique_new_links = [link for link in new_links if link not in saved_links]

with open('sample.txt', 'a', encoding='utf-8') as file:
    if unique_new_links:
        for link in unique_new_links:
            file.write(link + '\n')
            print('saved links found')
    
    else:
        print('duplicate not found')

saved links found
saved links found
saved links found


In [ ]:
for x in contaier:
    links = 'https://www.yelp.com' + x.get('href')
    
    data_links.append(links)
    print(links)

https://www.yelp.com/adredir?ad_business_id=2rvzTYaiXlqtBe5ZhNDiXA&campaign_id=213171001&click_origin=search_results&placement=vertical_0&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Ftopgolf-burlingame-4&request_id=92bfbf659fab57af&signature=f33faadbe59b63789fa45031581ad56c8d8e542a84787f2a604f53d3759617e7&slot=0
https://www.yelp.com/adredir?ad_business_id=HdZLo3v4zNVpzx1CQjg5MA&campaign_id=211872868&click_origin=search_results&placement=vertical_0&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fcapital-one-caf%25C3%25A9-san-francisco-3&request_id=92bfbf659fab57af&signature=c164666bf389a93890a62ac54bbf25592661e5fdfa6c2dee6ebb314fe46b7b2e&slot=1
https://www.yelp.com/adredir?ad_business_id=WKBAkKgeqJ9aCoexvpNUeA&campaign_id=213549189&click_origin=search_results&placement=vertical_0&placement_slot=0&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fstar-india-san-francisco-5&request_id=92bfbf659fab57af&signature=b670cc8dee7e7f2a1765d85bdab04c34b5b50e3135f

In [ ]:
sample_url = 'https://www.yelp.com/biz/hwa-mi-won-san-francisco?osq=Restaurants'

driver.get(sample_url)
# phone number 
driver._wait_loaded(2.1)

soup = fromstring(driver.html)

title_tag = soup.xpath('//div[contains(@class,"headingLight")]/h1')
title_text = title_tag[0].text.strip() if title_tag else 'None'
print(title_text)
phoneTag = soup.xpath('//div[@class="y-css-8x4us"]//p[text()="Phone number"]//following-sibling::p')
phone_number = phoneTag[0].text.strip() if phoneTag else 'phone not available'
print(phone_number)
# business site 
websiteTag = soup.xpath('//div[@class="y-css-8x4us"]//p[text()="Business website"]//following-sibling::p/a')
website_url = websiteTag[0].get('href').strip() if websiteTag else 'site not found'
print(website_url)
addressTag = soup.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
address = addressTag[0].text_content().strip() if addressTag else 'site not found'
print(address)

title_tag = soup.xpath('//div[contains(@class,"headingLight")]/h1')
title_text = title_tag[0].text.strip() if title_tag else 'None'
print(title_text)

Hwa Mi Won


In [ ]:
# phone number 
driver._wait_loaded(2.1)

soup = fromstring(driver.html)

title_tag = soup.xpath('//div[contains(@class,"headingLight")]/h1')
title_text = title_tag[0].text.strip() if title_tag else 'None'
print(title_text)
phoneTag = soup.xpath('//div[@class="y-css-8x4us"]//p[text()="Phone number"]//following-sibling::p')
phone_number = phoneTag[0].text.strip() if phoneTag else 'phone not available'
print(phone_number)
# business site 
websiteTag = soup.xpath('//div[@class="y-css-8x4us"]//p[text()="Business website"]//following-sibling::p/a')
website_url = websiteTag[0].get('href').strip() if websiteTag else 'site not found'
print(website_url)
addressTag = soup.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
address = addressTag[0].text_content().strip() if addressTag else 'site not found'
print(address)

(415) 742-4220


In [ ]:
# business site 
websiteTag = soup.xpath('//div[@class="y-css-8x4us"]//p[text()="Business website"]//following-sibling::p/a')
website_url = websiteTag[0].get('href').strip() if websiteTag else 'site not found'
print(website_url)
addressTag = soup.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
address = addressTag[0].text_content().strip() if addressTag else 'site not found'
print(address)

site not found


In [9]:
'//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p'

addressTag = soup.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
address = addressTag[0].text_content().strip() if addressTag else 'site not found'
print(address)

6314 Geary Blvd San Francisco, CA 94121


In [ ]:
url = 'https://www.drissionpage.cn/'

driver.get(url)
time.sleep(3)

scroll_page = driver.ele('//body')


In [ ]:
def read_urls(f):
    with open(f,'r',encoding='utf-8') as x:return [i.strip() for i in x if i.strip()]

def scrape_yelp_page(p,u):
    try:
        p.get(u);p._wait_loaded(3)
        s=fromstring(p.html)
        t=s.xpath('//div[contains(@class,"headingLight")]/h1')
        n=t[0].text.strip() if t else 'N/A'
        ph=s.xpath('//div[@class="y-css-8x4us"]//p[text()="Phone number"]/following-sibling::p')
        phn=ph[0].text.strip() if ph else 'N/A'
        w=s.xpath('//div[@class="y-css-8x4us"]//p[text()="Business website"]/following-sibling::p/a')
        ws=w[0].get('href').strip() if w else 'N/A'
        a=s.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
        ad=a[0].text_content().strip() if a else 'N/A'
        return {'Business Name':n,'Phone Number':phn,'Website':ws,'Address':ad,'URL':u}
    except Exception as e:
        print(f"Error scraping {u}:{e}")
        return {'Business Name':'Error','Phone Number':'Error','Website':'Error','Address':'Error','URL':u}

def save_to_csv(d,f='yelp_data.csv'):
    if not d:return
    k=d[0].keys()
    with open(f,'w',newline='',encoding='utf-8') as x:
        w=csv.DictWriter(x,fieldnames=k)
        w.writeheader();w.writerows(d)
    print(f"Saved to {f}")